In [1]:
import os
os.environ["XLA_FLAGS"] = '--xla_force_host_platform_device_count=10 --xla_cpu_enable_fast_math=true'

import jax
print(jax.devices())

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")

import numpy as np
import jax.numpy as jnp
import jax 
from jax import grad, vmap
jax.config.update("jax_enable_x64", True)


import matplotlib.pyplot as plt

import scipy.interpolate as interp
import scipy.integrate as integ
import scipy.linalg as sla

import fisher_jim_tgr_2par as lib
import pycbc.conversions

import astropy.units as u
from astropy import constants as const

Ms = (u.Msun * const.G / const.c**3 ).si.value
from datetime import datetime
datestr = datetime.now().strftime('%m-%d-%y')

import matplotlib as mpl
from matplotlib.legend_handler import HandlerLine2D, HandlerPatch

def reset_matplotlib():
    # Reset all matplotlib settings to defaults
    mpl.rcdefaults()

    # Create a default handler map and update it globally
    default_handler_map = {
        mpl.lines.Line2D: HandlerLine2D(numpoints=1),
        mpl.patches.Patch: HandlerPatch()
    }

    # Update the default handler map globally
    mpl.legend.Legend.update_default_handler_map(default_handler_map)

# Call this function at the start of your script to globally reset settings
reset_matplotlib()


An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


[CpuDevice(id=0), CpuDevice(id=1), CpuDevice(id=2), CpuDevice(id=3), CpuDevice(id=4), CpuDevice(id=5), CpuDevice(id=6), CpuDevice(id=7), CpuDevice(id=8), CpuDevice(id=9)]


In [2]:
# Ground stuff
n_freq = 2000
fmin = 10.
fmax = 1000.
freqs = jnp.logspace(jnp.log10(fmin), jnp.log10(fmax), num = int(n_freq))


full_str = ["M_c", "eta", "d_L", "ra", "dec", "iota", "psi", "t_c", "phase_c", "zco", "rng", "Mc_source", "snr_H1", "snr_L1", "snr_V1", "snr_t"]
names = full_str[0:9]
# data = np.loadtxt("data/events.txt")
data = jnp.array(np.loadtxt("data/5-year-data-O3/5-year-data-O3-events.txt"))
# data = jnp.array(nnp.loadtxt("data/5-year-data-CE/5-year-data-CE-events.txt"))

psd = lib.read_mag(freqs, "curves/o3_l1.txt")**2
# psd = lib.read_mag(freqs, "../curves/ce1.txt")**2

data_150914 = np.array(data[0])
data_150914[0] = 30
q  = 0.79
data_150914[1] = q/(1+q)**2
data_150914[2] = 390

idx = '150914'
if idx=='150914':
    dat = jnp.array(data_150914)

red_param = dict(zip(names, jnp.array(dat).T))

In [3]:
from jimgw.detector import H1, L1, V1
from jimgw.waveform import RippleIMRPhenomPv2, RippleIMRPhenomD
import jax
import jax.numpy as jnp
from jax import grad, vmap

jax.config.update("jax_enable_x64", True)
jax.config.update("jax_debug_nans", True)
# Assuming 'waveform' is defined elsewhere, or you can import it
waveform = RippleIMRPhenomPv2(f_ref=20)
# waveform = RippleIMRPhenomD(f_ref=20)

def get_h_slow(x, f, detector):
    # Set default parameters
    

    ff = jnp.array([f])
    h_sky = waveform(ff, x)
    align_time = jnp.exp(-1j * 2 * jnp.pi * ff * (x['epoch'] + x['t_c']))
    signal = detector.fd_response(ff, h_sky, x) * align_time
    return signal[0]

# Create generic functions for each detector
def get_h_nojit(x, f, detector):
    return vmap(lambda f_single: get_h_slow(x, f_single, detector))(f)



red_param = dict(zip(names, jnp.array(dat).T))
epsilon = jnp.array(1e-6)
for key in ['s1_x', 's1_y', 's1_z', 's2_x', 's2_y', 's2_z']:
    red_param[key] = epsilon
zero_vector = jnp.array(0.)
for key in ['gmst', 'epoch']:
    red_param[key] = zero_vector

x = red_param.copy()
# x['s1_z'] = 0.2
gr_param_diff = names + ['s1_z', 's1_x']

get_h1 = jax.jit(lambda x : get_h_nojit(x, freqs, H1))
get_h2 = jax.jit(lambda x : get_h_nojit(x, freqs, L1))
get_h3 = jax.jit(lambda x : get_h_nojit(x, freqs, V1))


get_h = lambda x : {'H1': get_h1(x), 'L1': get_h2(x), 'V1': get_h3(x)}


h = get_h(x)

keys = list(x.keys())
idx_diff = tuple(i for i, key in enumerate(keys) if key in gr_param_diff)

def f_wrapped(*args, freqs = None, det= None):
    x = dict(zip(keys, args)) 
    return get_h_nojit(x, freqs, H1)

get_dh1 = jax.jit(jax.jacfwd(lambda *x: f_wrapped(*x, freqs = freqs, det = H1), argnums = idx_diff))
get_dh2 = jax.jit(jax.jacfwd(lambda *x: f_wrapped(*x, freqs = freqs, det = H1), argnums = idx_diff))
get_dh3 = jax.jit(jax.jacfwd(lambda *x: f_wrapped(*x, freqs = freqs, det = H1), argnums = idx_diff))

def get_dh_gr(x):
    xvalues = list(x.values())

    dh = {'H1': dict(zip(gr_param_diff, get_dh1(*xvalues))) , 'L1': dict(zip(gr_param_diff, get_dh2(*xvalues))), 'V1': dict(zip(gr_param_diff, get_dh3(*xvalues)))}
    return dh

dh = get_dh_gr(x)

##  Inner Product Minimum Working Ex

In [5]:
paramx = ['M_c', 'eta', 'd_L', 'ra', 'dec', 'iota', 'psi', 't_c', 'phase_c']
idx_x = {paramx[i] : i for i in range(len(paramx))}
# %timeit compute_bias(dh["H1"], h["H1"], psd, freqs, idx_x)
log_flag =  {paramx[i] : 0 for i in range(len(paramx))}; log_flag["M_c"] = 1; log_flag["d_L"] = 1
logmult = {paramx[i]: (x[paramx[i]] if log_flag[paramx[i]] else 1) for i in range(len(paramx))}



def innprodslow(hf1, hf2, psd, freqs):
    integrand = (jnp.conj(hf1) * hf2 + hf1 * jnp.conj(hf2)) / psd
    prod = 2. * jnp.trapezoid(integrand, freqs)
    return prod

freqs = jnp.array(freqs, dtype=jnp.float64)
psd = jnp.array(psd, dtype=jnp.float64)
innprodfast = jax.jit(innprodslow)

dh_dim  = {key: logmult[key] * dh["H1"][key] for key in logmult}
i1 = "d_L"; i2 = 't_c'
hf1 = dh_dim[i1].copy(); hf2 = dh_dim[i2].copy()
innprodslow(hf1,hf2, psd, freqs), innprodfast(hf1,hf2, psd, freqs)

(Array(-1.81415985e-12+0.j, dtype=complex128),
 Array(-2.12628272e-12+0.j, dtype=complex128))

## Fisher Matrix Demo

fish slow uses the slow inner product with no jit

fish fast uses the fast inner product with jit

In [10]:
def fishslow(freqs, dh, par, idx_par, psd, log_flag):
    n_pt = len(freqs)
    n_dof = len(idx_par)

    dh_arr = jnp.zeros([n_dof, n_pt], dtype=jnp.complex128)

    # Convert idx_par to a list for static looping
    idx_list = list(idx_par.keys())
    for idx in idx_list:
        idx_position = idx_par[idx]
        dh_arr = dh_arr.at[idx_position, :].set(dh[idx])

        # Use jax.lax.cond for conditional multiplication
        dh_arr = dh_arr.at[idx_position, :].set(
            jax.lax.cond(
                log_flag[idx],
                lambda x: x * par[idx],
                lambda x: x,
                dh_arr[idx_position, :]
            )
        )

    gamma = jnp.zeros([n_dof, n_dof], dtype=jnp.float64)

    # Use static loops
    for i in range(n_dof):
        for j in range(i, n_dof):
            gamma = gamma.at[i, j].set(
                jnp.real(innprodslow(dh_arr[i, :], dh_arr[j, :], psd, freqs))
            )
        for j in range(i):
            gamma = gamma.at[i, j].set(jnp.conj(gamma[j, i]))

    return gamma

def fishfast(freqs, dh, par, idx_par, psd, log_flag):
    n_pt = len(freqs)
    n_dof = len(idx_par)

    dh_arr = jnp.zeros([n_dof, n_pt], dtype=jnp.complex128)

    # Convert idx_par to a list for static looping
    idx_list = list(idx_par.keys())
    for idx in idx_list:
        idx_position = idx_par[idx]
        dh_arr = dh_arr.at[idx_position, :].set(dh[idx])

        # Use jax.lax.cond for conditional multiplication
        dh_arr = dh_arr.at[idx_position, :].set(
            jax.lax.cond(
                log_flag[idx],
                lambda x: x * par[idx],
                lambda x: x,
                dh_arr[idx_position, :]
            )
        )

    gamma = jnp.zeros([n_dof, n_dof], dtype=jnp.float64)

    # Use static loops
    for i in range(n_dof):
        for j in range(i, n_dof):
            gamma = gamma.at[i, j].set(
                jnp.real(innprodfast(dh_arr[i, :], dh_arr[j, :], psd, freqs))
            )
        for j in range(i):
            gamma = gamma.at[i, j].set(jnp.conj(gamma[j, i]))

    return gamma

In [11]:
import pandas as pd 

display(pd.DataFrame(fishslow(freqs, dh["H1"], x, idx_x, psd, log_flag) ,paramx,paramx))
pd.DataFrame(np.abs(fishslow(freqs, dh["H1"], x, idx_x, psd, log_flag)- fishfast(freqs, dh["H1"], x, idx_x, psd, log_flag))/fishslow(freqs, dh["H1"], x, idx_x, psd, log_flag),paramx,paramx)

,M_c,eta,d_L,ra,dec,iota,psi,t_c,phase_c
M_c,1.941117e+05,-1.449215e+06,-5.526458e+02,3.480738e+04,6.248083e+04,28.508156,-16661.577969,-3.821259e+06,17634.043381
eta,-1.449215e+06,1.502091e+07,-1.260883e+03,-4.783218e+05,-1.023653e+06,4749.994527,189497.146149,6.040239e+07,-200777.338549
d_L,-5.526458e+02,-1.260883e+03,6.912320e+02,-4.456370e+02,-1.819569e+02,-464.401857,19.028877,-1.814160e-12,0.000149
ra,3.480738e+04,-4.783218e+05,-4.456370e+02,2.112128e+04,5.059044e+04,162.432783,-6663.681278,-2.901309e+06,7046.060381
dec,6.248083e+04,-1.023653e+06,-1.819569e+02,5.059044e+04,1.270890e+05,-183.354449,-14845.701722,-7.248934e+06,15721.231998
iota,2.850816e+01,4.749995e+03,-4.644019e+02,1.624328e+02,-1.833544e+02,313.051593,37.952096,1.786213e+04,-53.747032
psi,-1.666158e+04,1.894971e+05,1.902888e+01,-6.663681e+03,-1.484570e+04,37.952096,2464.400518,8.674248e+05,-2610.065618
t_c,-3.821259e+06,6.040239e+07,-1.814160e-12,-2.901309e+06,-7.248934e+06,17862.134437,867424.827852,4.142045e+08,-918891.554292
phase_c,1.763404e+04,-2.007773e+05,1.488438e-04,7.046060e+03,1.572123e+04,-53.747032,-2610.065618,-9.188916e+05,2764.928385


,M_c,eta,d_L,ra,dec,iota,psi,t_c,phase_c
M_c,0.0,-0.000000e+00,-0.000000e+00,0.000000e+00,0.0,0.000000e+00,-0.000000e+00,-0.000000,0.000000e+00
eta,-0.0,0.000000e+00,-1.803290e-16,-0.000000e+00,-0.0,0.000000e+00,0.000000e+00,0.000000,-0.000000e+00
d_L,-0.0,-1.803290e-16,0.000000e+00,-0.000000e+00,-0.0,-0.000000e+00,0.000000e+00,-0.172048,4.395080e-12
ra,0.0,-0.000000e+00,-0.000000e+00,0.000000e+00,0.0,1.749752e-16,-1.364853e-16,-0.000000,0.000000e+00
dec,0.0,-0.000000e+00,-0.000000e+00,0.000000e+00,0.0,-0.000000e+00,-0.000000e+00,-0.000000,0.000000e+00
iota,0.0,0.000000e+00,-0.000000e+00,1.749752e-16,-0.0,0.000000e+00,0.000000e+00,0.000000,-0.000000e+00
psi,-0.0,0.000000e+00,0.000000e+00,-1.364853e-16,-0.0,0.000000e+00,0.000000e+00,0.000000,-0.000000e+00
t_c,-0.0,0.000000e+00,-1.720482e-01,-0.000000e+00,-0.0,0.000000e+00,0.000000e+00,0.000000,-0.000000e+00
phase_c,0.0,-0.000000e+00,4.395080e-12,0.000000e+00,0.0,-0.000000e+00,-0.000000e+00,-0.000000,0.000000e+00


In [7]:
dh

NameError: name 'dh' is not defined

* First entry is 